In [0]:
!pip install tensorflow-gpu==2.0.0-beta0
import tensorflow as tf
import numpy as np
import os

import re
import nltk
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set([stemmer.stem(word) for word in stopwords.words('spanish')])
stopwords.remove('si')
stopwords.remove('no')
stopwords.remove('eres')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
class QA():
  def __init__(self, contexto_previo, contexto_posterior, intencion, preguntas, respuestas):
    self.contexto_previo = contexto_previo
    self.contexto_posterior = contexto_posterior
    self.intencion = intencion # Obligatorio
    self.preguntas = preguntas
    self.respuestas = respuestas

data = [
    QA('', 
       '',
       '',
       [''],
       ['No entendí tu consulta.']),
    QA('', 
       '', 
       'saludo',
       ['hola', 'hey como estas', 'hay alguien ahí'], 
       ['hola como estas', 'dime en que puedo ayudarte', 'estoy para servirte', 'en que te puedo ayudar']),
    QA('', 
       '',
       'descanso',
       ['que es descanso'], 
       ['despejar tu mente, recargar energias']
    ),
    QA('',
       '',
       'importancia',
       ['por que debo dormir'], 
       ['es muy importante para tu salud, puedes tener algunos problemas de salud']),
    QA('', 
       'presentacion',
       'presentacion',
       ['que eres', 'que haces', 'cual es tu funcion'],
       ['soy un chatbot programado para ayudarte']),
        QA('presentacion',
           '',
           'ayuda',
          ['ayudarme en que'],
          ['identificar problemas de descanso']),
    QA('', 
       '', 
       'agradecimiento',
       ['gracias', 'muchas gracias', 'gracias por tu ayuda', 'eres el mejor', 'me ayudaste mucho', 'gracias por la informacion'], 
       ['Feliz de poder ayudarte :).']),
    QA('', 
       '', 
       'despedida',
       ['chau y gracias', 'nos vemos luego', 'hasta pronto', 'adios', 'hasta luego'], 
       ['hasta pronto', 'nos vemos luego', 'espero haberte sido util']),
    QA('', 
       '',
       'sueño',
      ['tengo problemas de sueño', 'no duermo bien', 'por que debo dormir'],
      ['es muy importante para tu salud, puedes tener algunos problemas de salud']
      ),
    QA('', 
       '',
       'enfermedad',
      ['que enfermedades se originan por falta de sueño', 'quiero saber si tengo alguna enfermedad'],
      ['diabetes, hipertension arterial, obesisdad, depresion, ansiedad', 'dime tus sintomas']),
    QA('cuestionario', 
       'cuestionario',
       'afirmacion',
      ['si', 'claro', 'es verdad', 'afirmativo', 'siempre', 'acertaste'],
      []),
    QA('cuestionario', 
       'cuestionario',
       'negacion',
      ['no', 'claro que no', 'es mentira', 'negativo', 'nunca', 'fallaste'],
      []),
    QA('', 
       'diabetes',
       'diabetes',
      ['quiero saber si tengo diabetes', 'como se si tengo diabetes', 'tendre diabetes? ayudame'],
      ['Empezemos por un cuestionario, te parece bien?']),
        QA('diabetes', 
           'cuestionario',
           'cuestionario',
          ['si', 'claro', 'por supuesto', 'ok', 'como quieras', 'prosigue', 'afirmativo', 'como gustes', ' seria lo mejor', 'vamos', 'gracias'],
          []),
        QA('diabetes', 
          'no_cuestionario',
          'no_cuestionario',
          ['no', 'claro que no', 'por supuesto que no', 'no quiero', 'detente', 'negativo', 'no me gusta', 'no seria lo mejor', 'no gracias'],
          ['Ok como gustes, puedes realizarlo cuando quieras.'])
]

cuestionario = [
    '¿Has notado que últimamente te da más sed?',
    '¿Sientes más apetito que antes?',
    '¿Tienes fatiga?',
    '¿Tienes visión borrosa?',
    '¿Te han salido llagas en el cuerpo?',
    '¿Tienes sobrepeso?',
    '¿Sueles hacer a menudo actividad física?',
    '¿Sueles hacer a menudo algún deporte?',
    '¿Tienes zonas oscuras en tu piel?'
]

In [0]:
def limpiarTexto(text):
  text = text.lower()                                                                # convertir texto a minúsculas
  text = re.sub('[^a-zá-ú0-9ñÑ\s]+',' ',text)                                        # obtener solo caracteres y numeros
#   text = [stemmer.stem(w) for w in text.split()]                                     # remover tildes y estemizar
  text = [stemmer.stem(w) for w in text.split() if stemmer.stem(w) not in stopwords] # remover tildes, estemizar, remover stopwords
  return ' '.join(text)

for qa in data:
  qa.preguntas = [limpiarTexto(pregunta) for pregunta in qa.preguntas]

idx_a_intencion = list(set([qa.intencion for qa in data]))
intencion_a_idx = {t:i for i, t in enumerate(idx_a_intencion)}

preguntas = [pregunta for qa in data for pregunta in qa.preguntas]
idx_a_palabra = set([palabra for pregunta in preguntas for palabra in pregunta.split()])
idx_a_palabra.add('')
palabra_a_idx = {p:i for i, p in enumerate(idx_a_palabra)}

print(idx_a_intencion)
print(idx_a_palabra)

['', 'despedida', 'presentacion', 'descanso', 'no_cuestionario', 'importancia', 'sueño', 'negacion', 'saludo', 'afirmacion', 'agradecimiento', 'diabetes', 'ayuda', 'enfermedad', 'cuestionario']
{'', 'dorm', 'quier', 'pront', 'bien', 'diabet', 'sueñ', 'fall', 'problem', 'chau', 'sab', 'acert', 'ok', 'ayudam', 'hol', 'eres', 'clar', 'deb', 'siempr', 'funcion', 'hey', 'graci', 'duerm', 'enfermedad', 'vem', 'descans', 'supuest', 'ahi', 'hac', 'verd', 'algui', 'falt', 'prosig', 'adi', 'origin', 'gust', 'ayud', 'mentir', 'seri', 'no', 'detent', 'si', 'nunc', 'mejor', 'enfermed', 'vam', 'negat', 'informacion', 'lueg', 'afirm'}


In [0]:
# padding final
tam_secuencia = 10
pregunta_a_intencion = np.array([[pregunta, qa.intencion] for qa in data for pregunta in qa.preguntas])

x_train = [[palabra_a_idx[palabra] for palabra in pt[0].split()] for pt in pregunta_a_intencion]
x_train = tf.convert_to_tensor(np.array([np.pad(xt, (0, tam_secuencia - len(xt)), 'constant') for xt in x_train]), dtype=tf.int32)
y_train = tf.convert_to_tensor([intencion_a_idx[pt[1]] for pt in pregunta_a_intencion], dtype=tf.int32)

In [0]:
def build_model(tam_vocabulario, tam_intenciones, embedding_dim, batch_size, rnn_units):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(tam_vocabulario, embedding_dim, batch_input_shape=[batch_size, None]),
#       tf.keras.layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
#       tf.keras.layers.LSTM(rnn_units, return_sequences=False, recurrent_initializer='glorot_uniform'),
      tf.keras.layers.GRU(rnn_units, return_sequences=False, recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(tam_intenciones)
  ])
  return model
  
model = build_model(tam_vocabulario=len(idx_a_palabra), tam_intenciones=len(idx_a_intencion), embedding_dim=128, batch_size=len(pregunta_a_intencion), rnn_units=200)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (61, None, 128)           6400      
_________________________________________________________________
gru (GRU)                    (61, 200)                 198000    
_________________________________________________________________
dense (Dense)                (61, 15)                  3015      
Total params: 207,415
Trainable params: 207,415
Non-trainable params: 0
_________________________________________________________________


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam',loss=loss)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_prefix,
   save_weights_only=True)

In [0]:
EPOCHS=500
history = model.fit(x_train, y_train, epochs=EPOCHS, callbacks=[checkpoint_callback], verbose=True)

Train on 61 samples
Epoch 1/500
61/61 [==============================] - 0s 8ms/sample - loss: 2.7077
Epoch 2/500
61/61 [==============================] - 0s 845us/sample - loss: 2.6859
Epoch 3/500
61/61 [==============================] - 0s 736us/sample - loss: 2.6642
Epoch 4/500
61/61 [==============================] - 0s 892us/sample - loss: 2.6412
Epoch 5/500
61/61 [==============================] - 0s 927us/sample - loss: 2.6158
Epoch 6/500
61/61 [==============================] - 0s 903us/sample - loss: 2.5873
Epoch 7/500
61/61 [==============================] - 0s 931us/sample - loss: 2.5552
Epoch 8/500
61/61 [==============================] - 0s 847us/sample - loss: 2.5210
Epoch 9/500
61/61 [==============================] - 0s 833us/sample - loss: 2.4897
Epoch 10/500
61/61 [==============================] - 0s 783us/sample - loss: 2.4731
Epoch 11/500
61/61 [==============================] - 0s 704us/sample - loss: 2.4787
Epoch 12/500
61/61 [==============================] - 0s

In [0]:
model = build_model(tam_vocabulario=len(idx_a_palabra), tam_intenciones=len(idx_a_intencion), embedding_dim=128, batch_size=1, rnn_units=200)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            6400      
_________________________________________________________________
gru_1 (GRU)                  (None, 200)               198000    
_________________________________________________________________
dense_1 (Dense)              (None, 15)                3015      
Total params: 207,415
Trainable params: 207,415
Non-trainable params: 0
_________________________________________________________________


In [0]:
def obtenerQA(intencion):
  for qa in data:
    if qa.intencion == intencion:
      return qa
  return None

def responder(contexto_previo, pregunta):
  pregunta = limpiarTexto(pregunta)
  x = [palabra_a_idx[palabra] for palabra in pregunta.split() if palabra in idx_a_palabra]
  x = np.pad(x, (0, tam_secuencia - len(x)), 'constant')
  predictions = model(tf.expand_dims(x, 0))
  
  max_val = 0 # criterio
  best_qa = None
  
  for i, p in enumerate(predictions.numpy()[0]):
    qa = obtenerQA(idx_a_intencion[i])
    
    if qa.contexto_previo == contexto_previo or qa.intencion == '':
      #print(qa.contexto_previo, qa.intencion, p)
      if p>max_val:
        max_val = p
        best_qa = qa
    
  return best_qa

def iniciar_conversacion():
  consulta = ''
  contexto_previo = ''
  
  cuestionario_index = 0
  cuestionario_activo = False
  cuestionario_diabetes = 0
  
  while consulta != 'exit':
    consulta = input()
    qa = responder(contexto_previo, consulta)
    contexto_previo = qa.contexto_posterior
    
    if cuestionario_activo == False and contexto_previo == 'cuestionario':
      cuestionario_index = 0
      cuestionario_activo = True
      cuestionario_diabetes = 0
    
    if cuestionario_activo == True:
      if qa.intencion == 'afirmacion':
        cuestionario_diabetes += 1
        cuestionario_index += 1
      elif qa.intencion == 'negacion':
        cuestionario_index += 1
      else:
        contexto_previo = 'cuestionario' # evitamos el cambio de contexto por una mala entrada
        
    if cuestionario_activo == True:
      if(cuestionario_index >= len(cuestionario)):
        cuestionario_activo = False
        contexto_previo = ''
        print('Resultados:', cuestionario_diabetes)
      else:
        print(cuestionario[cuestionario_index])
    else:
      print(qa.respuestas)
      
iniciar_conversacion()

hola amigo
['hola como estas', 'dime en que puedo ayudarte', 'estoy para servirte', 'en que te puedo ayudar']
por que es importante dormir? :)
['despejar tu mente, recargar energias']
ah ok
['No entendí tu consulta.']
que eres un bot?
['soy un chatbot programado para ayudarte']
ayudarme a que???
['identificar problemas de descanso']
que enfermedades puedes detectar
['diabetes, hipertension arterial, obesisdad, depresion, ansiedad', 'dime tus sintomas']
como se si tengo diabetes?
['Empezemos por un cuestionario, te parece bien?']
ok dale
¿Has notado que últimamente te da más sed?
si
¿Sientes más apetito que antes?
no
¿Tienes fatiga?
no
¿Tienes visión borrosa?
no
¿Te han salido llagas en el cuerpo?
si
¿Tienes sobrepeso?
no
¿Sueles hacer a menudo actividad física?
no
¿Sueles hacer a menudo algún deporte?
si
¿Tienes zonas oscuras en tu piel?
si
Resultados: 4
muchas gracias :)
['Feliz de poder ayudarte :).']
nos vemos, chauuu
['Feliz de poder ayudarte :).']
nos vemos adios
['hasta pronto', 